In [9]:
from pulp import *
import numpy as np
import pandas as pd
model = LpProblem('MMA',LpMaximize)


fighters = pd.read_csv('C:\\Users\\liamf\\Downloads\\dks.csv')   #########
n = 5
numFights = len(fighters.filter(regex=("figh.*")).columns)

results = []
total_points = {}
cost = {}
number_of_fighters = {}
lock = {}
x = {}
fights= {} 
equations = {}

for i in range(1,numFights+1):
    var_name = "fight%d" % i 
    fights[var_name] = {}
    
for i, fighter in fighters.iterrows():
    var_name = 'x' + str(i) # Create variable name
    decision_var = LpVariable(var_name, cat='Binary') # Initialize Variables
    
    x[i] = decision_var 
    
    total_points[decision_var] = fighter["Points"] # Create PPG Dictionary
    cost[decision_var] = fighter["Salary"] # Create Cost Dictionary
    
    for i in fights:
        fights[i][decision_var] = fighter[i]
    lock[decision_var] = fighter["lock"]
    
    number_of_fighters[decision_var] = 1.0
    
objective_function = LpAffineExpression(total_points)
total_cost = LpAffineExpression(cost)
total_fighters = LpAffineExpression(number_of_fighters)
lock_eq = LpAffineExpression(lock)

for i in fights:
    var_name = i + "_eq"
    equations[var_name] = LpAffineExpression(fights[i])
    
model += objective_function
model += (total_cost <= 50000)
model += (total_fighters <= 6)
model += (total_fighters >= 6)
model += (lock_eq >=1)

for i in equations:
    model += (equations[i] <=1)
    
model.solve()

while len(results)<n:
    if model.solve() ==1:
        selection = []
        high = model.objective.value()
        fighters["is_drafted"] = 0.0
        for var in x.values():
            fighters.iloc[int(var.name[1:]),(len(fighters.columns)-1)] = var.varValue
        
        selection = fighters[fighters["is_drafted"] == 1.0]['ID']
            
            #if var.value() == 1.0:
                #selection.append(var.name)
            
        results.append(selection.values)
        
        model = LpProblem('MMA',LpMaximize)
        model += objective_function
        model += (total_cost <= 50000)
        model += (total_fighters <= 6)
        model += (lock_eq >=1)
        for i in equations:
            model += (equations[i] <=1)
        model += (objective_function <= (high - 0.01))
        model.solve()

In [10]:
for i in results:
    print(i)

[18970953 18970960 18970947 18970958 18970952 18970956]
[18970953 18970960 18970947 18970958 18970952 18970943]
[18970953 18970944 18970947 18970958 18970952 18970945]
[18970953 18970944 18970960 18970958 18970952 18970945]
[18970953 18970944 18970947 18970958 18970959 18970956]


In [7]:
len(results)

19